In [3]:
import re
import yaml
from pathlib import Path
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
from crewai import LLM

load_dotenv()

True

In [4]:
llm = LLM(
    model="openai/gpt-4", # call model by provider/model_name
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

In [5]:

# Use Path for file locations
current_dir = Path.cwd()
agents_config_path = current_dir / "config" / "agents.yaml"
tasks_config_path = current_dir / "config" / "tasks.yaml"

# Load YAML configuration files
with open(agents_config_path, "r") as file:
    agents_config = yaml.safe_load(file)

with open(tasks_config_path, "r") as file:
    tasks_config = yaml.safe_load(file)

## Define Agents
spamfilter = Agent(
    llm=llm,
    config=agents_config["spamfilter"], allow_delegation=False, verbose=True
)

analyst = Agent(config=agents_config["analyst"], allow_delegation=False, verbose=True)

scriptwriter = Agent(
    llm=llm,
    config=agents_config["scriptwriter"], allow_delegation=False, verbose=True
)

formatter = Agent(
    llm=llm,
    config=agents_config["formatter"], allow_delegation=False, verbose=True
)


scorer = Agent(config=agents_config["scorer"], allow_delegation=False, verbose=True)

In [13]:
discussion = """From: keith@cco.caltech.edu (Keith Allan Schneider)
Subject: Re: <Political Atheists?
Organization: California Institute of Technology, Pasadena
Lines: 50
NNTP-Posting-Host: punisher.caltech.edu

bobbe@vice.ICO.TEK.COM (Robert Beauchaine) writes:

>>I think that about 70% (or so) people approve of the
>>death penalty, even realizing all of its shortcomings.  Doesn't this make
>>it reasonable?  Or are *you* the sole judge of reasonability?
>Aside from revenge, what merits do you find in capital punishment?

Are we talking about me, or the majority of the people that support it?
Anyway, I think that "revenge" or "fairness" is why most people are in
favor of the punishment.  If a murderer is going to be punished, people
that think that he should "get what he deserves."  Most people wouldn't
think it would be fair for the murderer to live, while his victim died.

>Revenge?  Petty and pathetic.

Perhaps you think that it is petty and pathetic, but your views are in the
minority.

>We have a local televised hot topic talk show that very recently
>did a segment on capital punishment.  Each and every advocate of
>the use of this portion of our system of "jurisprudence" cited the
>main reason for supporting it:  "That bastard deserved it".  True
>human compassion, forgiveness, and sympathy.

Where are we required to have compassion, forgiveness, and sympathy?  If
someone wrongs me, I will take great lengths to make sure that his advantage
is removed, or a similar situation is forced upon him.  If someone kills
another, then we can apply the golden rule and kill this person in turn.
Is not our entire moral system based on such a concept?

Or, are you stating that human life is sacred, somehow, and that it should
never be violated?  This would sound like some sort of religious view.
 
>>I mean, how reasonable is imprisonment, really, when you think about it?
>>Sure, the person could be released if found innocent, but you still
>>can't undo the imiprisonment that was served.  Perhaps we shouldn't
>>imprision people if we could watch them closely instead.  The cost would
>>probably be similar, especially if we just implanted some sort of
>>electronic device.
>Would you rather be alive in prison or dead in the chair?  

Once a criminal has committed a murder, his desires are irrelevant.

And, you still have not answered my question.  If you are concerned about
the death penalty due to the possibility of the execution of an innocent,
then why isn't this same concern shared with imprisonment.  Shouldn't we,
by your logic, administer as minimum as punishment as possible, to avoid
violating the liberty or happiness of an innocent person?

keith
"""

In [ ]:
# Filter out spam and vulgar posts
task0 = Task(
    description=tasks_config["task0"]["description"],
    expected_output=tasks_config["task0"]["expected_output"],
    agent=spamfilter,
)

# process post with a crew of agents, ultimately delivering a well formatted dialogue
task1 = Task(
    description=tasks_config["task1"]["description"],
    expected_output=tasks_config["task1"]["expected_output"],
    agent=analyst,
)

task2 = Task(
    description=tasks_config["task2"]["description"],
    expected_output=tasks_config["task2"]["expected_output"],
    agent=scriptwriter,
)

task3 = Task(
    description=tasks_config["task3"]["description"],
    expected_output=tasks_config["task3"]["expected_output"],
    agent=formatter,
)
crew = Crew(
    agents=[analyst, scriptwriter, formatter],
    tasks=[task1, task2, task3],
    verbose=True,  # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
    process=Process.sequential,  # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

In [18]:


result = crew.kickoff(inputs={  "discussion": discussion })

# get rid of directions and actions between brackets, eg: (smiling)
script = re.sub(r"\(.*?\)", "", result.raw)

print("===================== end result from crew ===================================")
print(script)
print("===================== score ==================================================")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6cefc408-fd38-46d5-92db-f9e007f5873c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: analyse                                                                                                 │
│                                                                                                                 │
│  Task: Analyse in much detail the following discussion: ### DISCUSSION: {From: keith@cco.caltech.edu (Keith     │
│  Allan Schneider)                                                                                               │
│  Subject: Re: <Political Atheists?                                                                              │
│  Organization: California Institute of Technology, Pasadena                                                     │
│  Lines: 50                                                                                                      │
│  NNTP-Posting-Host: punisher.caltech.edu                                                                        │
│                                                                                                                 │
│  bobbe@vice.ICO.TEK.COM (Robert Beauchaine) writes:                                                             │
│                                                                                                                 │
│  >>I think that about 70% (or so) people approve of the                                                         │
│  >>death penalty, even realizing all of its shortcomings.  Doesn't this make                                    │
│  >>it reasonable?  Or are *you* the sole judge of reasonability?                                                │
│  >Aside from revenge, what merits do you find in capital punishment?                                            │
│                                                                                                                 │
│  Are we talking about me, or the majority of the people that support it?                                        │
│  Anyway, I think that "revenge" or "fairness" is why most people are in                                         │
│  favor of the punishment.  If a murderer is going to be punished, people                                        │
│  that think that he should "get what he deserves."  Most people wouldn't                                        │
│  think it would be fair for the murderer to live, while his victim died.                                        │
│                                                                                                                 │
│  >Revenge?  Petty and pathetic.                                                                                 │
│                                                                                                                 │
│  Perhaps you think that it is petty and pathetic, but your views are in the                                     │
│  minority.                                                                                                      │
│                                                                                                                 │
│  >We have a local televised hot topic talk show that very recently                                              │
│  >did a segment on capital punishment.  Each and every advocate of                                              │
│  >the use of this portion of our system of "jurisprudence" cited the                                            │
│  >main reason for supporting it:  "That bastard deserved it".  True                                             │
│  >human compassion, forgiveness, and sympathy.                                                                  │
│                                                        

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: analyse                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Robert Beauchaine's Initial Argument**: Robert begins by stating that approximately 70% of people         │
│  support the death penalty, questioning if this majority viewpoint can be deemed reasonable, or if only a       │
│  select few can judge validity. He challenges Keith to identify additional merits of capital punishment beyond  │
│  a sense of revenge.                                                                                            │
│                                                                                                                 │
│  2. **Keith Schneider's Response on Public Opinion and Revenge**: Keith acknowledges that the support for the   │
│  death penalty largely stems from sentiments of revenge and fairness. He expresses a common belief that         │
│  murderers "should get what they deserve," emphasizing that many people find it unjust for a murderer to        │
│  continue living while their victim is dead.                                                                    │
│                                                                                                                 │
│  3. **Keith on Revenge Being Petty**: Following Keith's mention of revenge, Robert argues that viewing capital  │
│  punishment through that lens—considering it as a vehicle for revenge—is "petty and pathetic."                  │
│                                                                                                                 │
│  4. **Keith's Defense of Majority Opinions**: Keith defends the popular perspective by implying that Robert's   │
│  viewpoint is a minority stance, further entrenching the notion that societal norms around fairness justify     │
│  the death penalty.                                                                                             │
│                                                                                                                 │
│  5. **Discussion on Compassion and Justice**: Robert refers to a recent televised talk show where advocates of  │
│  capital punishment shared their reasoning centered on the belief that the offender "deserved" their fate. He   │
│  questions the societal obligation for compassion, suggesting that for serious wrongs like murder, punitive     │
│  measures are justified—indicating a moral stance that supports revenge as a necessary aspect of justice.       │
│                                                                                                                 │
│  6. **Keith's Challenge on Morality**: Keith pushes back on the idea of compassion in justice, suggesting that  │
│  a moral system reflective of reciprocity (the Golden Rule) supports the execution of murderers. He hints that  │
│  if human life is deemed sacred, such views might stem from religious beliefs.                                  │
│                                                                                                                 │
│  7. **Imprisonment vs. Death Penalty**: In a philosophical turn, Robert questions the rationale behind          │
│  continued imprisonment of criminals, advocating for alternatives such as monitoring technology. He queries     │
│  Keith on the moral implications of preferring death over life imprisonment, especially if concerns about       │
│  wrongful punishment are involved.                     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f5b77112-7feb-49e7-8079-9bb0eb53dbf8                                                                     │
│  Agent: analyse                                                                                                 │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: scriptwriter                                                                                            │
│                                                                                                                 │
│  Task: Create a dialogue heavy screenplay from the discussion, between two persons. Do NOT write                │
│  parentheticals. Leave out wrylies. You MUST SKIP directional notes.                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: formatter                                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Robert Beauchaine's Initial Argument:                                                                       │
│    Look, seventy percent of people support the death penalty. Can we call that reasonable, or are only select   │
│  viewpoints valid? What's beyond the vengeance in capital punishment, Keith?                                    │
│                                                                                                                 │
│    ## Keith Schneider's Response on Public Opinion and Revenge:                                                 │
│    It's about fairness, Robert. People believe murderers should get what they deserve. If someone takes a       │
│  life, how can it be fair for them to continue living while their victim is dead?                               │
│                                                                                                                 │
│    ## Robert Beauchaine on Revenge Being Petty:                                                                 │
│    Considering capital punishment as a vehicle for revenge is petty and pathetic.                               │
│                                                                                                                 │
│    ## Keith Schneider's Defense of Majority Opinions:                                                           │
│    The popular                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9bc10aa8-91ee-4235-8a73-4a95d78d9c76                                                                     │
│  Agent: formatter                                                                                               │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6cefc408-fd38-46d5-92db-f9e007f5873c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Robert Beauchaine's Initial Argument:                                                         │
│    Look, seventy percent of people support the death penalty. Can we call that reasonable, or are only select   │
│  viewpoints valid? What's beyond the vengeance in capital punishment, Keith?                                    │
│                                                                                                                 │
│    ## Keith Schneider's Response on Public Opinion and Revenge:                                                 │
│    It's about fairness, Robert. People believe murderers should get what they deserve. If someone takes a       │
│  life, how can it be fair for them to continue living while their victim is dead?                               │
│                                                                                                                 │
│    ## Robert Beauchaine on Revenge Being Petty:                                                                 │
│    Considering capital punishment as a vehicle for revenge is petty and pathetic.                               │
│                                                                                                                 │
│    ## Keith Schneider's Defense of Majority Opinions:                                                           │
│    The popular                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): ===================== end result from crew ===================================
## Robert Beauchaine's Initial Argument:
  Look, seventy percent of people support the death penalty. Can we call that reasonable, or are only select viewpoints valid? What's beyond the vengeance in capital punishment, Keith?
         
  ## Keith Schneider's Response on Public Opinion and Revenge:
  It's about fairness, Robert. People believe murderers should get what they deserve. If someone takes a life, how can it be fair for them to continue living while their victim is dead?
         
  ## Robert Beauchaine on Revenge Being Petty:
  Considering capital punishment as a vehicle for revenge is petty and pathetic.
         
  ## Keith Schneider's Defense of Majority Opinions:
  The popular
===================== score ==================================================


In [20]:
task4 = Task(
    description=tasks_config["task4"]["description"],
    expected_output=tasks_config["task4"]["expected_output"],
    agent=scorer,
)

In [21]:
scorer.execute_task(
    task=task4,
    context=f"script: = {script}")

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: scorer                                                                                                  │
│                                                                                                                 │
│  Task: Score the following script: ### SCRIPT: {{script}}                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: scorer                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Score: 6                                                                                                       │
│                                                                                                                 │
│  Clarity: The dialogue is generally clear, but some terms could potentially confuse certain audiences (e.g.,    │
│  "vehicle for revenge").                                                                                        │
│  Relevance: Each participant stays on topic, discussing views on the death penalty and public opinion, which    │
│  contributes to the conversation's purpose.                                                                     │
│  Conciseness: The statements are relatively concise; however, some sentences could be trimmed for better        │
│  efficiency.                                                                                                    │
│  Politeness: There is a lack of politeness in Robert's description of capital punishment as "petty and          │
│  pathetic," which could lead to a breakdown in respectful dialogue.                                             │
│  Engagement: The participants engage with each other's points, but the tone escalates into an argument, which   │
│  could reduce engagement.                                                                                       │
│  Flow: The exchange has a decent flow, although the abruptness of Robert's statement could disrupt the          │
│  progression of ideas.                                                                                          │
│  Coherence: The dialogue is coherent overall, making logical sense within the context of the debate on capital  │
│  punishment.                                                                                                    │
│  Responsiveness: Both participants address each other's points, but Robert’s retort could be seen as            │
│  dismissive rather than a direct response, indicating a lapse in responsiveness.                                │
│  Language Use: The language is mostly appropriate; however, the phrase "petty and pathetic" lacks constructive  │
│  language.                                                                                                      │
│  Emotional Intelligence: Both participants seem invested in their viewpoints, but Robert’s language may         │
│  indicate a lack of sensitivity toward differing opinions and the emotional weight of the subject matter.       │
│                                                                                                                 │
│  Overall, while the script has several strengths, particularly in relevance and coherence, it suffers from      │
│  issues of politeness and emotional intelligence. This balance brings the scoring to a 6, indicating that it's  │
│  an average dialogue with notable weaknesses.                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

'Score: 6  \n\nClarity: The dialogue is generally clear, but some terms could potentially confuse certain audiences (e.g., "vehicle for revenge").  \nRelevance: Each participant stays on topic, discussing views on the death penalty and public opinion, which contributes to the conversation\'s purpose.  \nConciseness: The statements are relatively concise; however, some sentences could be trimmed for better efficiency.  \nPoliteness: There is a lack of politeness in Robert\'s description of capital punishment as "petty and pathetic," which could lead to a breakdown in respectful dialogue.  \nEngagement: The participants engage with each other\'s points, but the tone escalates into an argument, which could reduce engagement.  \nFlow: The exchange has a decent flow, although the abruptness of Robert\'s statement could disrupt the progression of ideas.  \nCoherence: The dialogue is coherent overall, making logical sense within the context of the debate on capital punishment.  \nResponsivene